# **Implémentation du modèle**
---

## Introduction

Contrairement au notebook "modèle", ici nous nous intéressons aux chocs de la politique monétaire américaine sur les pays européens.



Ce notebook contient les codes nécessaire à la mise en place du modèle. Le modèle est inspiré de celui de Iacoviello et Navarro : 

- La première étape consiste à identifier les chocs de politique monétaire, en prenant le résidu de la régression du taux d'intérêt sur l'écart d'inflation par rapport à la cible et l'output gap ou le chômage. En réalité Iacoviello et Navarro utilisent des lags et valeurs présentes de l'inflation, des spreads de crédit, du PIB et des lags des taux des fonds fédéraux.
Il nous manque donc l'output gap, l'inflation et le chômage de la zone euro
- La deuxième étape consiste à estimer l'impact de ces chocs sur l'activité économique (PIB, emploi ...) en régressant ces variables sur les chocs et des variables de contrôles (4 lags du PIB et des trends linéaires et quadratiques)

## Formatage des données

In [2]:
# import des données américaines

!pip install openpyxl 
import pandas as pd

sheets_to_load = ["Daily, Close", "Monthly", "Quarterly"]  # Noms des feuilles à importer
dfs = pd.read_excel("Données_extraites/data_US.xlsx", sheet_name=sheets_to_load, header=0)

# Accéder aux DataFrames importés
dfD = dfs["Daily, Close"]
dfM = dfs["Monthly"]
dfQ = dfs["Quarterly"]

# renommer les variables
dfD.columns = dfD.iloc[0] 
dfD = dfD[1:].reset_index(drop=True)

dfM.columns = dfM.iloc[0] 
dfM = dfM[1:].reset_index(drop=True)


# Il faut trimestrialiser les données (on prend la moyenne)

dfM["date"] = pd.to_datetime(dfM["date"])  # Convertir en datetime
dfM = dfM.set_index("date")  # Définir la colonne "date" comme index
dfM_Q = dfM.resample("Q").mean()  # Appliquer le resample par trimestre

dfD["date"] = pd.to_datetime(dfD["date"])  # Convertir en datetime
dfD = dfD.set_index("date")  # Définir la colonne "date" comme index
dfD_Q = dfD.resample("Q").mean()  # Appliquer le resample par trimestre

# harmoniser les dates
dfD_Q.index = dfD_Q.index.to_period("Q")  
dfM_Q.index = dfM_Q.index.to_period("Q")

dfQ=dfQ.rename(columns={"observation_date":"date"})
dfQ = dfQ.set_index("date")
dfQ.index = dfQ.index.to_period("Q")



/tmp/ipykernel_20799/499249651.py:26: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  dfM_Q = dfM.resample("Q").mean()  # Appliquer le resample par trimestre
/tmp/ipykernel_20799/499249651.py:30: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  dfD_Q = dfD.resample("Q").mean()  # Appliquer le resample par trimestre


In [4]:
import numpy as np
data_US = dfM_Q.merge(dfD_Q, on='date').merge(dfQ, on='date')

# Créer la variable d'inflation (sur 4 trimestres)
data_US["inflation"] = data_US["CPI"].pct_change(periods=4) * 100  # pct_change() donne la variation en pourcentage

# Créer une variable de temps et tendnce quadratique
data_US['time'] = range(1, len(data_US) + 1)
data_US['time_squared'] = data_US['time'] ** 2

# Créer 4 lags pour chaque variable
for col in data_US.columns:
    for lag in range(1, 5):  # Créer les lags 1, 2, 3, 4
        data_US[f"{col}_lag{lag}"] = data_US[col].shift(lag)


# Filtrer les colonnes contenant 'GDP'
gdp_columns = [col for col in data_US.columns if 'GDP' in col]

# Créer de nouvelles colonnes en log pour chaque colonne contenant 'GDP'
for col in gdp_columns:
    data_US[f"{col}_log"] = np.log(data_US[col])



/tmp/ipykernel_20799/3975620391.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_US["inflation"] = data_US["CPI"].pct_change(periods=4) * 100  # pct_change() donne la variation en pourcentage


In [5]:
data_US

,CPI,Policy rate,Industrial production,activité mensualisée,spread corporate,GDPC1,inflation,time,time_squared,CPI_lag1,...,time_lag4,time_squared_lag1,time_squared_lag2,time_squared_lag3,time_squared_lag4,GDPC1_log,GDPC1_lag1_log,GDPC1_lag2_log,GDPC1_lag3_log,GDPC1_lag4_log
date,,,,,,,,,,,,,,,,,,,,,
1996Q4,158.666667,5.28,76.830067,99.577753,0.6,12037.775,NaN,1,1,None,...,NaN,NaN,NaN,NaN,NaN,9.395805,NaN,NaN,NaN,NaN
1997Q1,159.633333,5.276667,78.286233,99.592907,0.591475,12115.472,NaN,2,4,158.666667,...,NaN,1.0,NaN,NaN,NaN,9.402239,9.395805,NaN,NaN,NaN
1997Q2,160.0,5.523333,79.403833,99.823247,0.584615,12317.221,NaN,3,9,159.633333,...,NaN,4.0,1.0,NaN,NaN,9.418754,9.402239,9.395805,NaN,NaN
1997Q3,160.8,5.533333,81.2606,100.017223,0.568308,12471.010,NaN,4,16,160.0,...,NaN,9.0,4.0,1.0,NaN,9.431162,9.418754,9.402239,9.395805,NaN
1997Q4,161.666667,5.506667,83.325533,100.0244,0.667273,12577.495,1.890756,5,25,160.8,...,1.0,16.0,9.0,4.0,1.0,9.439664,9.431162,9.418754,9.402239,9.395805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023Q4,308.158333,5.33,102.6986,100.44725,1.194923,22960.600,3.236199,109,11881,306.042333,...,105.0,11664.0,11449.0,11236.0,11025.0,10.041535,10.033679,10.023021,10.016970,10.010073
2024Q1,310.974333,5.33,102.242767,NaN,0.987969,23053.545,3.247987,110,12100,308.158333,...,106.0,11881.0,11664.0,11449.0,11236.0,10.045575,10.041535,10.033679,10.023021,10.016970
2024Q2,313.095667,5.33,102.8633,NaN,0.914394,23223.906,3.187395,111,12321,310.974333,...,107.0,12100.0,11881.0,11664.0,11449.0,10.052937,10.045575,10.041535,10.033679,10.023021


## Détermination des chocs

In [7]:
data_US.columns

Index(['CPI', 'Policy rate', 'Industrial production', 'activité mensualisée',
       'spread corporate', 'GDPC1', 'inflation', 'time', 'time_squared',
       'CPI_lag1', 'CPI_lag2', 'CPI_lag3', 'CPI_lag4', 'Policy rate_lag1',
       'Policy rate_lag2', 'Policy rate_lag3', 'Policy rate_lag4',
       'Industrial production_lag1', 'Industrial production_lag2',
       'Industrial production_lag3', 'Industrial production_lag4',
       'activité mensualisée_lag1', 'activité mensualisée_lag2',
       'activité mensualisée_lag3', 'activité mensualisée_lag4',
       'spread corporate_lag1', 'spread corporate_lag2',
       'spread corporate_lag3', 'spread corporate_lag4', 'GDPC1_lag1',
       'GDPC1_lag2', 'GDPC1_lag3', 'GDPC1_lag4', 'inflation_lag1',
       'inflation_lag2', 'inflation_lag3', 'inflation_lag4', 'time_lag1',
       'time_lag2', 'time_lag3', 'time_lag4', 'time_squared_lag1',
       'time_squared_lag2', 'time_squared_lag3', 'time_squared_lag4',
       'GDPC1_log', 'GDPC1_lag1_log',

In [13]:
import statsmodels.api as sm
# Supprimer les lignes avec NaN ou Inf
data_US_clean = data_US.dropna()
data_US_clean=data_US_clean.apply(pd.to_numeric, errors='coerce')
# Définition de Y (variable dépendante)
Y = data_US_clean['Policy rate']

# Sélectionner toutes les colonnes sauf 'key_rate' et 'TIME_PERIOD'
X = data_US_clean[['Policy rate_lag1','Policy rate_lag2', 'Policy rate_lag3', 'Policy rate_lag4',
'inflation','inflation_lag1','inflation_lag2', 'inflation_lag3', 'inflation_lag4',
'GDPC1_log', 'GDPC1_lag1_log', 'GDPC1_lag2_log', 'GDPC1_lag3_log','GDPC1_lag4_log',
'spread corporate','spread corporate_lag1', 'spread corporate_lag2','spread corporate_lag3', 'spread corporate_lag4',
'time_squared']]



# Ajouter une constante pour l'intercept dans la régression
X = sm.add_constant(X)

# Régression
model = sm.OLS(Y, X).fit()

# Extraire les résidus comme chocs monétaires
data_US_clean['monetary_shocks'] = model.resid

# Afficher le résumé de la régression
print(model.summary())
with open("regression_results.tex", "w") as f:
    f.write(model.summary().as_latex())
# Sauvegarder les résultats dans un fichier CSV
data_US_clean[['time', 'monetary_shocks']].to_csv('identified_monetary_shocks.csv', index=False)


                            OLS Regression Results                            
Dep. Variable:            Policy rate   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                     280.2
Date:                Sun, 23 Mar 2025   Prob (F-statistic):           1.58e-65
Time:                        16:31:17   Log-Likelihood:                 1.2524
No. Observations:                 101   AIC:                             39.50
Df Residuals:                      80   BIC:                             94.41
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.18